# Import the libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# combination of random oversampling and undersampling for imbalanced classification
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
import sklearn
import datetime as dt
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


# Import the dataset

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/train.csv')

In [5]:
df_test=pd.read_csv('/content/gdrive/MyDrive/test.csv')

In [6]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 621300 entries, 0 to 621299
Columns: 136 entries, SITE_ID to kurt_temperature_alarms_prev14d
dtypes: float64(118), int64(17), object(1)
memory usage: 644.7+ MB
None


In [4]:
target=df['aircon_sum_target_next14d']

# Count classes


In [5]:
# Count number of Fraud cases as a ratio of total transactions in the data
count = df['aircon_sum_target_next14d'].value_counts()
print(count)
print(count / len(df.index))

0    617717
1      3583
Name: aircon_sum_target_next14d, dtype: int64
0    0.994233
1    0.005767
Name: aircon_sum_target_next14d, dtype: float64


# Modity dataset to get rid of some columns




In [6]:
data1 = df.copy()

y=target

data1['DATE'] = pd.to_datetime(data1['DATE'])

data1.drop(labels='aircon_sum_target_next14d', axis=1, inplace=True)
data1.drop(columns=['GEOGRAPHIC_CLUSTER_K_0', 'GEOGRAPHIC_CLUSTER_K_1',
       'GEOGRAPHIC_CLUSTER_K_2', 'GEOGRAPHIC_CLUSTER_K_3',
       'GEOGRAPHIC_CLUSTER_K_4', 'GEOGRAPHIC_CLUSTER_K_5',
       'GEOGRAPHIC_CLUSTER_K_6', 'GEOGRAPHIC_CLUSTER_K_7',
       'GEOGRAPHIC_CLUSTER_K_8', 'GEOGRAPHIC_CLUSTER_K_9'], inplace=True)
data1.drop(columns='CELL_TYPE_TRP', inplace=True)

#data1

data2=data1.copy()
X=data1

data2['DATE']=data2['DATE'].map(dt.datetime.toordinal)

data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 621300 entries, 0 to 621299
Columns: 124 entries, SITE_ID to kurt_temperature_alarms_prev14d
dtypes: datetime64[ns](1), float64(118), int64(5)
memory usage: 587.8 MB


In [7]:
data3=data1[['N_TRANSPORTED_SITES', 'kurt_equipment_alarms_prev14d', 'kurt_power_alarms_prev14d', 'aircon_sum_wo_prev14d', 'kurt_ge_alarms_prev14d', 'kurt_temperature_alarms_prev14d', 'kurt_fire/smoke_alarms_prev14d', 'skew_equipment_alarms_prev14d', 'aircon_sum_wo_prev7d', 'mean_rain_mm_f_next14d', 'mean_humidity_f_next14d', 'equipment_sum_alarms_prev14d', 'mean_temperature_f_next14d', 'temperature_sum_alarms_prev14d', 'skew_temperature_alarms_prev14d', 'mean_wind_speed_f_next14d', 'max_rain_mm_f_next14d', 'min_humidity_prev7d', 'mean_humidity_prev7d', 'max_humidity_f_next14d', 'min_humidity_f_next14d', 'mean_temperature_f_next7d', 'mean_humidity_f_next7d', 'max_rain_mm_prev7d', 'min_humidity_f_next7d', 'temperature_mean_persistance_prev7d', 'mean_pressure_f_next14d', 'mean_rain_mm_f_next7d', 'mean_wind_speed_prev7d', 'mean_wind_speed_f_next7d']]
data3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 621300 entries, 0 to 621299
Data columns (total 30 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   N_TRANSPORTED_SITES                  621300 non-null  float64
 1   kurt_equipment_alarms_prev14d        621300 non-null  float64
 2   kurt_power_alarms_prev14d            621300 non-null  float64
 3   aircon_sum_wo_prev14d                621300 non-null  float64
 4   kurt_ge_alarms_prev14d               621300 non-null  float64
 5   kurt_temperature_alarms_prev14d      621300 non-null  float64
 6   kurt_fire/smoke_alarms_prev14d       621300 non-null  float64
 7   skew_equipment_alarms_prev14d        621300 non-null  float64
 8   aircon_sum_wo_prev7d                 621300 non-null  float64
 9   mean_rain_mm_f_next14d               621300 non-null  float64
 10  mean_humidity_f_next14d              621300 non-null  float64
 11  equipment_sum

# Final data set to use





In [8]:
x = data3.iloc[:,:].values
y = df['aircon_sum_target_next14d'].values
y

array([0, 0, 0, ..., 0, 0, 0])

Columns i used:
['N_TRANSPORTED_SITES', 'kurt_equipment_alarms_prev14d',
       'kurt_power_alarms_prev14d', 'aircon_sum_wo_prev14d',
       'kurt_ge_alarms_prev14d', 'kurt_temperature_alarms_prev14d',
       'kurt_fire/smoke_alarms_prev14d', 'skew_equipment_alarms_prev14d',
       'aircon_sum_wo_prev7d', 'mean_rain_mm_f_next14d',
       'mean_humidity_f_next14d', 'equipment_sum_alarms_prev14d',
       'mean_temperature_f_next14d', 'temperature_sum_alarms_prev14d',
       'skew_temperature_alarms_prev14d', 'mean_wind_speed_f_next14d',
       'max_rain_mm_f_next14d', 'min_humidity_prev7d', 'mean_humidity_prev7d',
       'max_humidity_f_next14d', 'min_humidity_f_next14d',
       'mean_temperature_f_next7d', 'mean_humidity_f_next7d',
       'max_rain_mm_prev7d', 'min_humidity_f_next7d',
       'temperature_mean_persistance_prev7d', 'mean_pressure_f_next14d',
       'mean_rain_mm_f_next7d', 'mean_wind_speed_prev7d',
       'mean_wind_speed_f_next7d']


# Training Random Forest

In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split (x,y, test_size=0.2, random_state=0)

In [14]:
# Training the model
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=10,criterion='entropy', random_state=0)
classifier.fit(x,y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [15]:
# Predict Y on the test set
y_pred = classifier.predict(x_test)

In [16]:
# Obtain the results from the classification report and confusion matrix 
from sklearn.metrics import  classification_report, confusion_matrix 

print('Classifcation report:\n', classification_report(y_test, y_pred))
conf_mat = confusion_matrix(y_true=y_test, y_pred=y_pred)
print('Confusion matrix:\n', conf_mat)

Classifcation report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    123573
           1       0.99      0.89      0.94       687

    accuracy                           1.00    124260
   macro avg       0.99      0.95      0.97    124260
weighted avg       1.00      1.00      1.00    124260

Confusion matrix:
 [[123566      7]
 [    74    613]]


# Undersampling
try this one, oversample one, SMOTE


In [17]:
# This is the pipeline module we need from imblearn
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline 

# Define which resampling method and which ML model to use in the pipeline
resampling = RandomUnderSampler()
model = RandomForestClassifier(n_estimators=10,criterion='entropy', random_state=0)

# Define the pipeline,and combine sampling method with the RF model
pipeline = Pipeline([('RandomUnderSampler', resampling), ('RF', model)])

pipeline.fit(x_train, y_train) 
predicted = pipeline.predict(x_test)

# Obtain the results from the classification report and confusion matrix 
print('Classifcation report:\n', classification_report(y_test, predicted))
conf_mat = confusion_matrix(y_true=y_test, y_pred=predicted)
print('Confusion matrix:\n', conf_mat)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Classifcation report:
               precision    recall  f1-score   support

           0       1.00      0.86      0.92    123573
           1       0.03      0.90      0.07       687

    accuracy                           0.86    124260
   macro avg       0.52      0.88      0.50    124260
weighted avg       0.99      0.86      0.92    124260

Confusion matrix:
 [[106243  17330]
 [    66    621]]


# Oversampling

In [18]:
# This is the pipeline module we need from imblearn
from imblearn.over_sampling import RandomOverSampler

# Define which resampling method and which ML model to use in the pipeline
resampling = RandomOverSampler()
model = RandomForestClassifier(n_estimators=10,criterion='entropy', random_state=0)

# Define the pipeline,and combine sampling method with the RF model
pipeline = Pipeline([('RandomOverSampler', resampling), ('RF', model)])

pipeline.fit(x_train, y_train) 
predicted = pipeline.predict(x_test)

# Obtain the results from the classification report and confusion matrix 
print('Classifcation report:\n', classification_report(y_test, predicted))
conf_mat = confusion_matrix(y_true=y_test, y_pred=predicted)
print('Confusion matrix:\n', conf_mat)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Classifcation report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    123573
           1       0.75      0.41      0.53       687

    accuracy                           1.00    124260
   macro avg       0.87      0.71      0.77    124260
weighted avg       1.00      1.00      1.00    124260

Confusion matrix:
 [[123477     96]
 [   403    284]]




# SMOTE 


In [19]:
# This is the pipeline module we need from imblearn
from imblearn.over_sampling import SMOTE
# Define which resampling method and which ML model to use in the pipeline
resampling = SMOTE(sampling_strategy='auto',random_state=0)
model = RandomForestClassifier(n_estimators=10,criterion='entropy', random_state=0)

# Define the pipeline, tell it to combine SMOTE with the RF model
pipeline = Pipeline([('SMOTE', resampling), ('RF', model)])

pipeline.fit(x_train, y_train) 
predicted = pipeline.predict(x_test)

# Obtain the results from the classification report and confusion matrix 
print('Classifcation report:\n', classification_report(y_test, predicted))
conf_mat = confusion_matrix(y_true=y_test, y_pred=predicted)
print('Confusion matrix:\n', conf_mat)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Classifcation report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    123573
           1       0.80      0.53      0.64       687

    accuracy                           1.00    124260
   macro avg       0.90      0.76      0.82    124260
weighted avg       1.00      1.00      1.00    124260

Confusion matrix:
 [[123485     88]
 [   326    361]]


# Under+Over sampling together



In [20]:
# define model
model = RandomForestClassifier()
# define resampling
over = RandomOverSampler(sampling_strategy=0.01)
under = RandomUnderSampler(sampling_strategy=0.6)
# define pipeline
pipeline = Pipeline(steps=[('o', over), ('u', under), ('m', model)])
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(pipeline, x, y, scoring='roc_auc', cv=cv, n_jobs=-1)
# summarize performance
print('Mean ROC AUC: %.3f' % mean(scores))
scores = cross_val_score(pipeline, x, y, scoring='f1', cv=cv, n_jobs=-1)
# summarize performance
print('f1: %.3f' % mean(scores))

Mean ROC AUC: 0.986
f1: 0.228


# SMOTE + ENN



In [ ]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN

sme = SMOTEENN(random_state=42)
X_res, y_res = sme.fit_resample(x, y)
print('Resampled dataset shape %s' % Counter(y_res))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split (X_res,y_res, test_size=0.2, random_state=0)
# Training the model
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=10,criterion='entropy', random_state=0)
classifier.fit(X_res,y_res)
# Predict Y on the test set
y_pred = classifier.predict(x_test)
# Obtain the results from the classification report and confusion matrix 
from sklearn.metrics import  classification_report, confusion_matrix 

print('Classifcation report:\n', classification_report(y_test, y_pred))
conf_mat = confusion_matrix(y_true=y_test, y_pred=y_pred)
print('Confusion matrix:\n', conf_mat)